In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
# Load your movie dataset
movies_df = pd.read_csv('/content/drive/MyDrive/tmdb_5000_movies.csv')
credits = pd.read_csv('/content/drive/MyDrive/tmdb_5000_credits.csv')

# Explore the dataset
movies_df.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [15]:
movies_df = movies_df.merge(credits,on='title')
movies_df.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [16]:
movies_df.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'movie_id', 'cast', 'crew'],
      dtype='object')

In [17]:
# Combine relevant features into a single column
movies_df['combined_features'] = movies_df['genres'] + " " + movies_df['keywords'] + " " + movies_df['cast'] + " " + movies_df['crew']

# Fill any missing values
movies_df = movies_df.fillna('')

# Check the updated DataFrame
movies_df.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew,combined_features
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""..."


In [9]:
# Vectorize Text Data
# Use CountVectorizer to convert text data into a matrix of token counts
count_vectorizer = CountVectorizer()
count_matrix = count_vectorizer.fit_transform(movies_df['combined_features'])


In [10]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [12]:
def recommend_movies(movie_title):
    movie_index = movies_df[movies_df['original_title'] == movie_title].index[0]
    similar_movies = list(enumerate(cosine_sim[movie_index]))

    # Sort movies based on similarity scores
    sorted_movies = sorted(similar_movies, key=lambda x: x[1], reverse=True)[1:]

    # Display top 5 recommended movies
    recommended_movies = [movies_df.iloc[movie[0]]['original_title'] for movie in sorted_movies[:5]]
    return recommended_movies

In [13]:
# Test the recommendation system
user_input_movie = "The Dark Knight Rises"
recommended_movies = recommend_movies(user_input_movie)

print(f"Recommended movies for {user_input_movie}:")
for i, movie in enumerate(recommended_movies, 1):
    print(f"{i}. {movie}")

Recommended movies for The Dark Knight Rises:
1. The Wolf of Wall Street
2. Batman Begins
3. Spider-Man
4. Spider-Man 2
5. Gone Girl


In [18]:
# Implement Genre-Based Recommendation Function
def recommend_movies_by_genre(user_input_genre):
    # Filter movies based on the entered genre
    genre_filter = movies_df['genres'].apply(lambda genres: user_input_genre.lower() in genres.lower())
    genre_movies = movies_df[genre_filter]

    if genre_movies.empty:
        return "No movies found in the specified genre."

    # Use the same recommendation function as before
    genre_count_matrix = count_vectorizer.transform(genre_movies['combined_features'])
    genre_cosine_sim = cosine_similarity(genre_count_matrix, count_matrix)

    # Get movie indices with highest similarity
    genre_similar_movies = list(enumerate(genre_cosine_sim.mean(axis=0)))
    sorted_genre_movies = sorted(genre_similar_movies, key=lambda x: x[1], reverse=True)[1:]

    # Display top 5 recommended movies
    recommended_genre_movies = [movies_df.iloc[movie[0]]['original_title'] for movie in sorted_genre_movies[:5]]
    return recommended_genre_movies

# Test the genre-based recommendation system
user_input_genre = "Action"
recommended_genre_movies = recommend_movies_by_genre(user_input_genre)

print(f"Recommended {user_input_genre} movies:")
for i, movie in enumerate(recommended_genre_movies, 1):
    print(f"{i}. {movie}")


Recommended Action movies:
1. The Hunger Games: Catching Fire
2. Batman Begins
3. Dances with Wolves
4. Der Untergang
5. The Rage: Carrie 2
